In [1]:
from model.LightGCN import *
from preprocess.AmazonBook import *
from evaluation.LightGCN_evaluation import *
from evaluation.LightGCN_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/amazon-book'
dataset = AmazonBook(path)

data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 0.001,
    'epochs': 1000,
    'num_layers': 2,
    'batch_size': 2048,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
}
model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)

# Split the dataset

In [2]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

# Retain LightGCN model

In [ ]:
config['epochs'] = 1000
retrain_lightgcn = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)
retrain_lightgcn, epoch_tracks, test_topks = lightgcn_eva(retrain_lightgcn, config, retain_data, device)

In [ ]:
# Test on the forget data
lightgcn_forget_data_eva(retrain_lightgcn, None, forget_data, config['num_users'], config['k'], config['batch_size'], device)

# Prompt Unlearning
## Case 1: Without Contrastive Loss and Regularization

In [3]:
# Define the model
teacher = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)
student = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"lightGCN_Amazon_Book_{config['epochs']}_Epochs.pt"))
student.load_state_dict(torch.load(f"lightGCN_Amazon_Book_{config['epochs']}_Epochs.pt"))

<All keys matched successfully>

In [ ]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 10
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt, epoch_tracks, test_topks = prompt_lightgcn_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_label_index', 'edge_index', 'x'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch: 001, Loss: 0.5510, HR@20: 0.0142, Recall@20: 0.0341, NDCG@20: 0.0257
Epoch: 002, Loss: 0.5083, HR@20: 0.0142, Recall@20: 0.0340, NDCG@20: 0.0257
Epoch: 003, Loss: 0.4508, HR@20: 0.0142, Recall@20: 0.0339, NDCG@20: 0.0257


KeyboardInterrupt: 

In [5]:
# Test on the forget data
lightgcn_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0268, Recall@20: 0.1552, NDCG@20: 0.1597


## Case 2: Without Contrastive Loss but with Regularization

In [6]:
# Setting the basic hyperparameters
config['contrastive_loss'] = False
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_lightgcn_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

Epoch: 001, Loss: 0.5698, HR@20: 0.0137, Recall@20: 0.0329, NDCG@20: 0.0805
Epoch: 002, Loss: 0.5404, HR@20: 0.0136, Recall@20: 0.0326, NDCG@20: 0.0799
Epoch: 003, Loss: 0.4984, HR@20: 0.0135, Recall@20: 0.0325, NDCG@20: 0.0794
Epoch: 004, Loss: 0.4570, HR@20: 0.0135, Recall@20: 0.0323, NDCG@20: 0.0789
Epoch: 005, Loss: 0.4293, HR@20: 0.0135, Recall@20: 0.0323, NDCG@20: 0.0788
Epoch: 006, Loss: 0.3971, HR@20: 0.0134, Recall@20: 0.0322, NDCG@20: 0.0787
Epoch: 007, Loss: 0.3756, HR@20: 0.0134, Recall@20: 0.0323, NDCG@20: 0.0786
Epoch: 008, Loss: 0.3582, HR@20: 0.0134, Recall@20: 0.0323, NDCG@20: 0.0786
Epoch: 009, Loss: 0.3388, HR@20: 0.0134, Recall@20: 0.0323, NDCG@20: 0.0786
Epoch: 010, Loss: 0.3256, HR@20: 0.0135, Recall@20: 0.0324, NDCG@20: 0.0786
Total time: 208.20s


In [7]:
# Test on the forget data
lightgcn_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0255, Recall@20: 0.1472, NDCG@20: 0.1534


## Case 3: With Contrastive Loss but without Regularization

In [ ]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 10
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
# Setting the basic hyperparameters
config['Contrastive_loss'] = True
config['regularization'] = False

student, prompt, epoch_tracks, test_topks = prompt_lightgcn_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

In [9]:
# Test on the forget data
lightgcn_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0243, Recall@20: 0.1375


## Case 4: With Contrastive Loss and Regularization

In [10]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_lightgcn_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

Epoch: 001, Loss: 0.6785, HR@20: 0.0138, Recall@20: 0.0330
Epoch: 002, Loss: 0.6717, HR@20: 0.0138, Recall@20: 0.0330
Epoch: 003, Loss: 0.6621, HR@20: 0.0138, Recall@20: 0.0331
Epoch: 004, Loss: 0.6501, HR@20: 0.0138, Recall@20: 0.0332
Epoch: 005, Loss: 0.6356, HR@20: 0.0138, Recall@20: 0.0332
Epoch: 006, Loss: 0.6257, HR@20: 0.0139, Recall@20: 0.0333
Epoch: 007, Loss: 0.6112, HR@20: 0.0139, Recall@20: 0.0334
Epoch: 008, Loss: 0.6016, HR@20: 0.0140, Recall@20: 0.0335
Epoch: 009, Loss: 0.5831, HR@20: 0.0140, Recall@20: 0.0335
Epoch: 010, Loss: 0.5750, HR@20: 0.0140, Recall@20: 0.0336
Epoch: 011, Loss: 0.5642, HR@20: 0.0140, Recall@20: 0.0336
Epoch: 012, Loss: 0.5614, HR@20: 0.0140, Recall@20: 0.0337
Epoch: 013, Loss: 0.5561, HR@20: 0.0140, Recall@20: 0.0337
Epoch: 014, Loss: 0.5462, HR@20: 0.0140, Recall@20: 0.0337
Epoch: 015, Loss: 0.5493, HR@20: 0.0141, Recall@20: 0.0338
Epoch: 016, Loss: 0.5459, HR@20: 0.0141, Recall@20: 0.0338
Epoch: 017, Loss: 0.5430, HR@20: 0.0141, Recall@20: 0.03

In [11]:
# Test on the forget data
lightgcn_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0253, Recall@20: 0.1448


# Additional Materials(Multiple Prompts)

In [12]:
config['tuning_type'] = 'gpf-plus'
config["number_p"] = 10 # The number of prompts
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_lightgcn_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

Epoch: 001, Loss: 0.6978, HR@20: 0.0139, Recall@20: 0.0333
Epoch: 002, Loss: 0.6803, HR@20: 0.0138, Recall@20: 0.0330
Epoch: 003, Loss: 0.6519, HR@20: 0.0137, Recall@20: 0.0328
Epoch: 004, Loss: 0.6215, HR@20: 0.0137, Recall@20: 0.0327
Epoch: 005, Loss: 0.6013, HR@20: 0.0136, Recall@20: 0.0326
Epoch: 006, Loss: 0.5859, HR@20: 0.0137, Recall@20: 0.0326
Epoch: 007, Loss: 0.5516, HR@20: 0.0137, Recall@20: 0.0327
Epoch: 008, Loss: 0.5351, HR@20: 0.0137, Recall@20: 0.0327
Epoch: 009, Loss: 0.5281, HR@20: 0.0138, Recall@20: 0.0328
Epoch: 010, Loss: 0.5232, HR@20: 0.0137, Recall@20: 0.0329
Epoch: 011, Loss: 0.4904, HR@20: 0.0138, Recall@20: 0.0330
Epoch: 012, Loss: 0.4740, HR@20: 0.0138, Recall@20: 0.0332
Epoch: 013, Loss: 0.4466, HR@20: 0.0139, Recall@20: 0.0333
Epoch: 014, Loss: 0.4162, HR@20: 0.0139, Recall@20: 0.0333
Epoch: 015, Loss: 0.4063, HR@20: 0.0138, Recall@20: 0.0332
Epoch: 016, Loss: 0.3676, HR@20: 0.0139, Recall@20: 0.0335
Epoch: 017, Loss: 0.3472, HR@20: 0.0139, Recall@20: 0.03

In [13]:
# Test on the forget data
lightgcn_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0255, Recall@20: 0.1462


In [14]:
# Test on the forget data
lightgcn_forget_data_eva(teacher, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0255, Recall@20: 0.1462
